In [ ]:
!pip install -q -U datasets kaleido

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from collections import defaultdict
from typing import List, Dict
import pandas as pd
import plotly.express as px
from datasets import load_dataset
import random
import numpy as np
from huggingface_hub import hf_hub_download
import os
import plotly.graph_objects as go
import json

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 9.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64"

In [ ]:
SQL_KEYWORDS = {
    "where", "join", "inner", "left", "right", "full",
    "on", "group", "by", "order", "having", "limit", "offset",
    "union", "distinct", "count", "sum", "avg", "min", "max",
    "asc", "desc", "as"
}

def extract_table_names(raw_prompt: str) -> set[str]:
    import re
    tbl = set()
    create_match = re.search(r"CREATE\s+TABLE\s+([A-Za-z_][A-Za-z0-9_]*)",
                             raw_prompt, flags=re.I)
    from_match   = re.search(r"\bFROM\s+([A-Za-z_][A-Za-z0-9_]*)",
                             raw_prompt, flags=re.I)
    if create_match:
        tbl.add(create_match.group(1).lower())
    if from_match:
        tbl.add(from_match.group(1).lower())
    return tbl

In [ ]:
class LogitLensProfiler:
    def __init__(self, model_name: str, device: str = None):
        self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")
        self.tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            output_hidden_states=True,
            return_dict=True,
        ).to(self.device)
        if self.tokenizer.pad_token_id is None:
            self.tokenizer.pad_token_id = self.tokenizer.eos_token_id

    def profile_example(self, prompt: str) -> dict[int, dict[str, list[float]]]:
        enc        = self.tokenizer(prompt, return_tensors="pt")
        input_ids  = enc["input_ids"][0].to(self.device)
        seq_len    = input_ids.size(0)

        table_set  = extract_table_names(prompt)
        scores: dict[int, dict[str, list[float]]] = {}

        for t in range(1, seq_len):
            prefix   = input_ids[:t].unsqueeze(0)
            true_id  = input_ids[t].item()
            tok_str  = self.tokenizer.decode([true_id]).lower().strip()

            if tok_str in SQL_KEYWORDS:
                cat = "keyword"
            elif tok_str in table_set:
                cat = "table"
            else:
                cat = "other"

            with torch.no_grad():
                hidden_states = self.model(prefix).hidden_states

            for L, h in enumerate(hidden_states):
                if L not in scores:
                    scores[L] = {c: [] for c in ("keyword", "table", "other")}

                h_t    = h[0, -1, :]
                logits = self.model.lm_head(h_t)
                prob   = torch.softmax(logits, dim=-1)[true_id].item()
                scores[L][cat].append(prob)

        return scores

In [ ]:
def plot_keyword_vs_table(
    layer2catprobs: dict[str, dict[int, dict[str, list[float]]]],
    *,
    title: str = "Keyword vs. Table‑name emergence",
    mode: str = "raw",
    baseline_layer: int = 0,
    zoom_first_n: int | None = None,
):
    rows = []
    for ex_name, layer_dict in layer2catprobs.items():
        baseline = {cat: np.nan for cat in ("keyword", "table")}
        if mode == "delta":
            baseline_cats = layer_dict.get(baseline_layer, {})
            for cat in ("keyword", "table"):
                vals = baseline_cats.get(cat, [])
                if vals:
                    baseline[cat] = np.mean(vals)

        for layer, cats in layer_dict.items():
            for cat in ("keyword", "table"):
                vals = cats.get(cat, [])
                if not vals:
                    continue
                mean_p = np.mean(vals)

                if mode == "logit":
                    eps = 1e-6
                    mean_p = np.clip(mean_p, eps, 1 - eps)
                    mean_p = np.log(mean_p / (1 - mean_p))
                elif mode == "delta":
                    if np.isnan(baseline[cat]):
                        continue
                    mean_p = mean_p - baseline[cat]

                rows.append(
                    dict(
                        Example=ex_name,
                        Layer=layer,
                        Category=cat.capitalize(),
                        Score=mean_p,
                    )
                )

    df = pd.DataFrame(rows)

    fig = px.line(
        df,
        x="Layer",
        y="Score",
        color="Category",
        line_dash="Example",
        markers=True,
        template="plotly_white",
    )
    fig.update_layout(
        title={"text": title, "x": 0.5, "font": {"size": 56}},
        yaxis_title={
            "raw": "P(next token = ground‑truth)",
            "logit": "logit probability",
            "delta": f"Δ P  vs. layer {baseline_layer}",
        }[mode],
        yaxis_title_font={"size": 56},
        xaxis_title_font={"size": 56},
        legend_title_font={"size": 56},
        legend_font={"size": 56},
        font=dict(size=56),
        xaxis=dict(tickfont=dict(size=56)),
        yaxis=dict(tickfont=dict(size=56)),
        legend_title="Token category",
        template="plotly_white",
    )
    fig.update_traces(showlegend=False)

    raw_color = (
        fig.data[0].line.color
        if hasattr(fig.data[0], "line")
        else fig.data[0].marker.color
    )
    smth_color = (
        fig.data[-1].line.color
        if hasattr(fig.data[1], "line")
        else fig.data[1].marker.color
    )

    fig.add_trace(
        go.Scatter(
            x=[None],
            y=[None],
            mode="lines",
            line=dict(color=raw_color, width=2),
            name="Tables",
        )
    )
    fig.add_trace(
        go.Scatter(
            x=[None],
            y=[None],
            mode="lines",
            line=dict(color=smth_color, width=2),
            name="SQL Keywords",
        )
    )
    fig.show()
    fig.write_image("keyword_vs_table.pdf", format="pdf", width=2400, height=900)

    if zoom_first_n is not None:
        df_zoom = df[df["Layer"] < zoom_first_n]
        if len(df_zoom):
            fig2 = px.line(
                df_zoom,
                x="Layer",
                y="Score",
                color="Category",
                line_dash="Example",
                markers=True,
                template="plotly_dark",
            )
            fig2.update_layout(
                title={"text": f"Zoom: first {zoom_first_n} layers", "x": 0.5, "font": {"size": 48}},
                yaxis_title=fig.layout.yaxis.title.text,
                yaxis_title_font={"size": 48},
                xaxis_title_font={"size": 48},
                legend_title_font={"size": 48},
                legend_font={"size": 48},
                font=dict(size=48),
                xaxis=dict(tickfont=dict(size=48)),
                yaxis=dict(tickfont=dict(size=48)),
                legend_title="Token category",
            )
            fig2.show()

In [ ]:
model_name = "withmartian/sql_interp_bm3_cs1_experiment_7.3"
profiler    = LogitLensProfiler(model_name)

tokenizer_config.json:   0%|          | 0.00/54.8k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/989 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:820: UserWarning:

`return_dict_in_generate` is NOT set to `True`, but `output_hidden_states` is. When `return_dict_in_generate` is not `True`, `output_hidden_states` is ignored.



generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

In [ ]:
ds = load_dataset("withmartian/cs1_dataset", split="test")

def build_prompt(row):
    return f"{row['create_statement']}\n{row['sql_statement']}"

N = 100
sample_indices = random.sample(range(len(ds)), N)
prompts = [build_prompt(ds[i]) for i in sample_indices]

README.md:   0%|          | 0.00/795 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/15.9M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/2.09M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/76500 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
all_cat_scores = {}
for i, p in enumerate(prompts):
    all_cat_scores[f"Example {i+1}"] = profiler.profile_example(p)

In [ ]:
with open('all_cat_scores_bm3_cs1.json', 'w') as f:
    json.dump(all_cat_scores, f, indent=2)

In [ ]:
# ONLY RUN TO LOAD RESULTS FROM JSON
with open('/content/all_cat_scores_bm3_cs3.json', 'r') as f:
    all_cat_scores = json.load(f)

In [ ]:
plot_keyword_vs_table(all_cat_scores,
                      title="", mode="logit")